In [14]:
import pandas as pd
import numpy as np
import json

df = pd.read_csv('../output/steam_app_data_raw.csv')
df = df[~df['name'].isna()]
df = df[~df['developer'].isna()]
df = df[~df['publisher'].isna()]
df.rename({"appid" : "id_app"}, axis=1, inplace=True)
df["price_usd"] = df["price"] / 100.0
df["initial_price_usd"] = df["initialprice"] / 100.0

In [ ]:
df["genre"] = df["genre"].apply(lambda x : str(x).split(','))
df_genres = pd.DataFrame(df.explode("genre")["genre"])
df_genres["genre"] = df_genres["genre"].apply(lambda x : x.strip())
df_genres = df_genres.drop_duplicates(subset="genre")
df_genres = df_genres[df_genres["genre"]!='nan']
df_genres = df_genres.reset_index(drop=True).reset_index(names='id_genre')

In [4]:
df_apps_genres = df.explode("genre")[["id_app", "genre"]].reset_index(drop=True)
df_apps_genres = df_apps_genres[df_apps_genres["genre"]!='nan']
df_apps_genres["genre"] = df_apps_genres["genre"].apply(lambda x : x.strip())
df_apps_genres = df_apps_genres.merge(df_genres, on='genre', how='left').drop('genre', axis=1)

In [213]:
df.drop(columns=['genre'], inplace=True)

In [18]:
def normalize_list_column(
        original_df: pd.DataFrame,
        id_column: str,
        list_column: str,
        new_table_id_column: str
    ) -> tuple:
        # Create a copy of original dataframe
        df = original_df.copy()

        # Create new table with unique values from that column
        df_new_table = pd.DataFrame(df.explode(list_column)[list_column])
        df_new_table[list_column] = df_new_table[list_column] .apply(lambda x : x.strip())
        df_new_table = df_new_table.drop_duplicates(subset=list_column)
        df_new_table = df_new_table[df_new_table[list_column]!='nan']
        df_new_table = df_new_table.reset_index(drop=True).reset_index(names=new_table_id_column)

        # Create new intermediate table that joins two tables
        df_intermediate_table = df.explode(list_column)[[id_column, list_column]].reset_index(drop=True)
        df_intermediate_table = df_intermediate_table[df_intermediate_table[list_column]!='nan']
        df_intermediate_table[list_column] = df_intermediate_table[list_column].apply(lambda x : x.strip())
        df_intermediate_table = df_intermediate_table.merge(df_new_table, on=list_column, how='left').drop(list_column, axis=1)

        # Drop list column from original table
        df.drop(columns=[list_column], inplace=True)

        # Return 3 dataframes in a tuple
        return df, df_new_table, df_intermediate_table

In [43]:
def transform_tags_column(
       col: str
    ) -> list:
        # Return empty list if no tags
        if not col:
            return []
        # Remove decades quotes (1990's)
        col = col.replace("0'", "0")
        # Remove quotes from abbreviations (Shoot 'Em Up)
        col = col.replace("Shoot 'Em Up", "Shoot Em Up")
        # Replace ' for " if JSON is in a bad format
        col = col.replace("{'", '{"')
        col = col.replace("':", '":')
        col = col.replace(", '", ', "')
        # Transform dict into list of tuples
        list_tags = []
        tags_data = dict(json.loads(col))
        for tag, count in tags_data.items():
            list_tags.append((tag, count))
        return list_tags

string = r'''{"2D Platformer": 52, "Action": 51, "Platformer": 46, "Shoot 'Em Up": 43, "Singleplayer": 40, "Casual": 39, "2D": 37, "Controller": 29, "Arcade": 27, "Pixel Graphics": 25, "Retro": 23, "Indie": 21, "Combat": 21, "Collectathon": 17, "Action-Adventure": 15, "Old School": 13}'''
transform_tags_column(string)

[('2D Platformer', 52),
 ('Action', 51),
 ('Platformer', 46),
 ('Shoot Em Up', 43),
 ('Singleplayer', 40),
 ('Casual', 39),
 ('2D', 37),
 ('Controller', 29),
 ('Arcade', 27),
 ('Pixel Graphics', 25),
 ('Retro', 23),
 ('Indie', 21),
 ('Combat', 21),
 ('Collectathon', 17),
 ('Action-Adventure', 15),
 ('Old School', 13)]

In [17]:
def normalize_json_column(
        original_df: pd.DataFrame,
        id_column: str,
        json_column: str,
        new_table_id_column: str
    ) -> tuple:
        # Create a copy of original dataframe
        df = original_df.copy()
        # 

        # Create new table with unique values from that column
        df_new_table = pd.DataFrame(df.explode(json_column)[json_column])
        df_new_table[json_column] = df_new_table[json_column] .apply(lambda x : x.strip())
        df_new_table = df_new_table.drop_duplicates(subset=json_column)
        df_new_table = df_new_table[df_new_table[json_column]!='nan']
        df_new_table = df_new_table.reset_index(drop=True).reset_index(names=new_table_id_column)

        # Create new intermediate table that joins two tables
        df_intermediate_table = df.explode(json_column)[[id_column, json_column]].reset_index(drop=True)
        df_intermediate_table = df_intermediate_table[df_intermediate_table[json_column]!='nan']
        df_intermediate_table[json_column] = df_intermediate_table[json_column].apply(lambda x : x.strip())
        df_intermediate_table = df_intermediate_table.merge(df_new_table, on=json_column, how='left').drop(json_column, axis=1)

        # Drop list column from original table
        df.drop(columns=[json_column], inplace=True)

        # Return 3 dataframes in a tuple
        return df, df_new_table, df_intermediate_table


In [20]:
string: str = "{'Sports': 57, 'Simulation': 54, 'Strategy': 51, 'Basketball': 43, '2D': 28, 'Singleplayer': 22, 'Indie': 21}"
json.loads(string)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [ ]:
df1, df2, df3 = normalize_json_column(
    original_df=df,
    id_column='id_app',
    json_column='tags',
    new_table_id_column='tag'
)

In [11]:
df2

,tag,tags
0,0,[]
1,1,"{'Sports': 57, 'Simulation': 54, 'Strategy': 5..."
2,2,"{'Casual': 73, 'Arcade': 47, 'e-sports': 44, '..."
3,3,"{'Casual': 68, 'Strategy': 61, 'Arcade': 39, '..."
4,4,"{'Pixel Graphics': 54, 'Cute': 47, 'Automation..."
...,...,...
62481,62481,"{'RPG': 82, 'Strategy': 72, 'JRPG': 59, 'Strat..."
62482,62482,"{'MOBA': 264, 'Third Person': 252, 'Strategy':..."
62483,62483,"{'Colorful': 380, 'Rogue-lite': 375, 'RPG': 36..."
62484,62484,"{'Early Access': 173, 'City Builder': 157, 'Hi..."


In [254]:
len(df3)

110086

In [214]:
def transform_languages_column(
       col: str
    ) -> str:
    # Chars to remove
    invalid_chars = ['strong', 'amp', '*', '&', 'lt;', 'gt;', ';', r'\/', '/', 'br']

    # Make column a list
    languages = col.split(',')
    for idx, lang in enumerate(languages):
        # Remove invalid chars
        for char in invalid_chars:
            lang = lang.replace(char, '')
        # Replace \r for ' - ' for languages like 'English\nInterface: English\nFull Audio: Flemish\nSubtitles'
        lang = lang.replace('\r\n', ' - ')
        # Remove whitespaces
        lang = lang.strip()
        # Reflect changes in the list
        languages[idx] = lang

    # Filter random Steam languages like '#lang_#lang_spanish*#lang_full_audio'
    for lang in languages:
        if lang.startswith('#'):
            languages.remove(lang)

    return languages

# string = "English&lt;strong&gt;*&lt;\/strong&gt;, German, French, Polish, Spanish - Spain&lt;br&gt;&lt;br&gt;&lt;strong&gt;*&lt;\/strong&gt; "
# transform_languages_column(string)

In [216]:
df["languages"] = df["languages"].apply(lambda x: transform_languages_column(str(x)))

In [217]:
df_apps, df_languages, df_apps_languages = normalize_list_column(
    original_df=df,
    id_column='id_app',
    list_column='languages',
    new_table_id_column='id_language'
)

In [218]:
df_languages.languages.values

array(['English', 'French', 'Italian', 'German', 'Spanish - Spain',
       'Japanese', 'Korean', 'Polish', 'Portuguese - Portugal', 'Russian',
       'Simplified Chinese', 'Traditional Chinese', 'Portuguese - Brazil',
       'Spanish - Latin America', 'Arabic', 'Bulgarian', 'Hungarian',
       'Vietnamese', 'Greek', 'Danish', 'Dutch', 'Norwegian', 'Romanian',
       'Thai', 'Turkish', 'Ukrainian', 'Finnish', 'Czech', 'Swedish',
       'Indonesian', 'Not supported', 'Hindi', 'Slovak', 'Bangla',
       'Galician', 'Malay', 'Catalan', 'Persian', 'Kazakh', 'Mongolian',
       'Khmer', 'Tamil', 'Filipino', 'Azerbaijani', 'Belarusian',
       'Bosnian', 'Croatian', 'Estonian', 'Irish', 'Icelandic',
       'Lithuanian', 'Nepali', 'Serbian', 'Latvian', 'Heew', 'Swahili',
       'Urdu', 'Portuguese', 'Welsh', 'Albanian', 'Georgian', 'Uzbek',
       'Basque', 'Luxembourgish', 'Afrikaans', 'Slovenian', 'Valencian',
       'Spanish', 'Scots', 'Portuguese-Brazil', 'Maori', 'Marathi',
       'Macedo

In [220]:
df_languages[df_languages["languages"]=='(all with full audio support)']

,id_language,languages
80,80,(all with full audio support)


In [221]:
df_apps_languages[df_apps_languages["id_language"]==80]

,id_app,id_language
291720,13000,80


In [32]:
df.sample(1)

,id_app,name,developer,publisher,score_rank,owners,average_forever,average_2weeks,median_forever,median_2weeks,ccu,price,initialprice,discount,tags,languages,genre,price_usd,initial_price_usd
55936,1251690,My Hole is a Mouth of Dirt,Yai Gameworks,Yai Gameworks,NaN,"0 .. 20,000",0,0,0,0,0,0.0,0.0,0.0,"{'Indie': 58, 'Adventure': 57, 'Violent': 34, ...",English,"Adventure, Indie",0.0,0.0
